In [13]:
!pip install -q unsloth==2025.4.7 datasets==3.5.1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
import torch
from trl import SFTTrainer
from datasets import load_dataset
from unsloth import FastLanguageModel
from unsloth import FastLanguageModel
from transformers import GenerationConfig
from transformers import TrainingArguments

In [15]:

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
from huggingface_hub import login



In [17]:
MAX_SEQ_LENGTH = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
  model_name="unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
  max_seq_length=MAX_SEQ_LENGTH,
  load_in_4bit=True,
  dtype=torch.bfloat16,
  )

==((====))==  Unsloth 2025.7.3: Fast Llama patching. Transformers: 4.53.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [18]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=[
    "q_proj", "k_proj", "v_proj", "up_proj",
    "down_proj", "o_proj", "gate_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth",
    random_state = 42,
    )

In [19]:
model.print_trainable_parameters()

trainable params: 11,272,192 || all params: 1,247,086,592 || trainable%: 0.9039


In [8]:
dataset_name = "5CD-AI/Vietnamese-Multi-turn-Chat-Alpaca"
raw_dataset = load_dataset(dataset_name, split="train")

Generating train split:   0%|          | 0/12697 [00:00<?, ? examples/s]

In [20]:
SYS_INSTRUCT = "Bạn là một trợ lý AI thân thiện, hãy trả lời bằng tiếng Việt."

def convert_to_chat_format(conversations):
    messages = [{"role": "system", "content": SYS_INSTRUCT}]
    for msg in conversations:
      role = "user" if msg["from"] == "human" else "assistant"
      messages.append({"role": role, "content": msg["value"]})
    return messages

def format_prompt(example):
    messages = convert_to_chat_format(example["conversations"])
    return {
      "text": tokenizer.apply_chat_template(
      messages,
      tokenize=False,
      add_generation_prompt=False
    )
    }

def tokenize_function(examples):
    return tokenizer(
    examples["text"],
    truncation=True,
    max_length=MAX_SEQ_LENGTH,
    padding="max_length",
    )

dataset = raw_dataset.map(format_prompt, remove_columns=raw_dataset.column_names)
dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/12697 [00:00<?, ? examples/s]

Map:   0%|          | 0/12697 [00:00<?, ? examples/s]

In [21]:
import os
os.environ['UNSLOTH_RETURN_LOGITS'] = '1'  # Yêu cầu Unsloth trả logits (hữu ích nếu muốn tính loss, hoặc debug)

# Cấu hình cho việc huấn luyện mô hình
training_args = TrainingArguments(
  per_device_train_batch_size=2,              # Batch size mỗi GPU (trong 1 bước)
  gradient_accumulation_steps=2,              # Tích lũy gradient trong 2 bước → batch thực tế = 2×2 = 4
  learning_rate=1e-4,                         # Learning rate ban đầu
  save_total_limit=4,                         # Chỉ giữ lại tối đa 4 checkpoint (xóa bản cũ)
  logging_steps=20,                           # Ghi log sau mỗi 20 bước huấn luyện
  output_dir="./checkpoint/llama3-1b-multi-conversation",  # Thư mục lưu checkpoint
  optim="paged_adamw_8bit",                   # Dùng optimizer 8bit (ít tốn VRAM hơn)
  lr_scheduler_type="cosine",                 # Dùng cosine decay cho learning rate
  warmup_ratio=0.05,                          # 5% số bước đầu để warmup learning rate
  save_strategy="steps",                      # Lưu mô hình sau mỗi N bước
  save_steps=50,                              # Lưu checkpoint sau mỗi 50 bước
  report_to="none",                           # Không gửi log đến hệ thống ngoài (ví dụ wandb)
  remove_unused_columns=True,                 # Bỏ cột không dùng trong dataset (tăng tốc huấn luyện)
  max_steps=800,                              # Chạy tối đa 400 bước huấn luyện
  bf16=True,                                  # Dùng bfloat16 nếu GPU hỗ trợ (nhanh và tiết kiệm RAM hơn float32)
)

# Khởi tạo Trainer để huấn luyện mô hình với SFT (Supervised Fine-Tuning)
trainer = SFTTrainer(
  model=model,                                # Mô hình đã gắn LoRA (từ FastLanguageModel)
  args=training_args,                         # Cấu hình huấn luyện
  train_dataset=dataset,                      # Dataset đã tokenize
  tokenizer=tokenizer,                        # Tokenizer tương ứng với mô hình
)

# Bắt đầu huấn luyện
trainer.train()


In [ ]:
# sau khi train co ca baser + lora . Cần merge lại trước khi save and push
model, tokenizer = FastLanguageModel.from_pretrained(
    "./checkpoint/llama3-1b-multi-conversation/checkpoint-400",
    max_seq_length=2048,
    load_in_4bit=True,
    dtype=torch.bfloat16,
)

==((====))==  Unsloth 2025.6.12: Fast Llama patching. Transformers: 4.53.0.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
model.push_to_hub_merged(
  "DATVO110/Llama-3.2-1B-Instruct-Chat-sft",
  commit_message="Merge weights to push to hub",
)


tokenizer.push_to_hub(
  "DATVO110/Llama-3.2-1B-Instruct-Chat-sft",
  commit_message="Push tokenizer to hub",
)

Unsloth: You're not saving a tokenizer as well?
You can do it separately via `tokenizer.push_to_hub(...)`


Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Unsloth: Merging weights into 16bit:   0%|          | 0/4 [00:00<?, ?it/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Uploading...:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [04:54<00:00, 73.62s/it]


README.md:   0%|          | 0.00/601 [00:00<?, ?B/s]

Uploading...:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
model_path="DATVO110/Llama-3.2-1B-Instruct-Chat-sft"
# prompt="Question: What is the capital of France?\nChoices:\nA. Berlin\nB. Paris\nC. Madrid\nD. Rome\nAnswer:"
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = model_path,
        max_seq_length = 2048,
        dtype = torch.bfloat16,
        load_in_4bit = True,
    )

# inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
# outputs = model.generate(**inputs, max_new_tokens=10)
# answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
# print(answer)

==((====))==  Unsloth 2025.6.12: Fast Llama patching. Transformers: 4.53.0.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [ ]:
generation_config = GenerationConfig(
  max_new_tokens=256,                 # Mô hình sẽ sinh tối đa 256 tokens mới (không tính input)
  temperature=1.0,                    # Độ ngẫu nhiên (1.0 là bình thường, càng thấp càng xác định)
  do_sample=False,                    # Không dùng sampling → sẽ dùng greedy decoding hoặc beam search
  num_return_sequences=1,            # Chỉ sinh ra 1 câu trả lời duy nhất
  pad_token_id=tokenizer.eos_token_id,  # Token dùng để pad nếu cần (dùng luôn EOS token cho an toàn)
  eos_token_id=tokenizer.eos_token_id,  # Khi gặp token này thì dừng sinh (End of Sequence)
  repetition_penalty=1.3             # Phạt lặp lại từ → tránh mô hình lặp câu (1.0 = không phạt, >1.0 = giảm lặp)
)

prompt = [
  {"role": "system", "content": (
  "Bạn là một trợ lý AI thân thiện, "
  "hãy trả lời bằng tiếng Việt."
  )},

  {"role": "user", "content": (
  "Hãy chỉnh sửa câu này để ngắn gọn hơn mà không mất đi ý nghĩa: "
  "\"Trận đấu là một thất bại nặng nề "
  "mặc dù thực tế là cả đội đã tập luyện trong nhiều tuần.\""
  )},

  {"role": "assistant", "content": (
  "Nhiều tuần huấn luyện của đội đã dẫn đến một thất bại nặng nề."
  )},

  {"role": "user", "content": (
  "Bạn có thể đề xuất một số chiến lược mà "
  "nhóm có thể sử dụng để cải thiện hiệu suất "
  "của họ trong trận đấu tiếp theo không?"
  )}
]





chat_text = tokenizer.apply_chat_template(
    prompt,
    add_generation_prompt=True,
    tokenize=False
)

inputs = tokenizer(
    chat_text,
    return_tensors="pt"            # Trả về định dạng tensor PyTorch
).to("cuda:0")                     # Đưa dữ liệu lên GPU để sinh kết quả nhanh


with torch.no_grad():             # Không cần tính gradient (inference)
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        generation_config=generation_config,   # Cấu hình sinh (max_tokens, temp, v.v.)
    )

output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

if "assistant\n" in output_text:
  answer = output_text.split("assistant\n")[-1].strip()
else:
  answer = output_text.strip()

print("Assistant reply:", answer)

Assistant reply: Chắc chắn! Dưới đây là một vài mẹo giúp các thành viên trong nhóm cải tiến khả năng và kết quả ở trò chơi sau: 1. Nhóm nên dành thời gian nghiên cứu đối thủ tiềm ẩn về phong cách đánh chặn, điểm mạnh cũng như nhược điểm của chúng ta nhằm phát triển kế hoạch thích hợp cho mình trước khi bước vào sân thi đấu. Điều quan trọng cần lưu ý đó là phải hiểu được mọi người xung quanh bạn đang làm gì vì điều này sẽ ảnh hưởng trực tiếp đến những lựa chọn cá nhân mà mỗi cầu thủ đưa ra trên sân cỏ hoặc phòng ngự. Nếu ai đó biết rằng anh ấy đang bị tấn công thì tất nhiên anh ấy muốn di chuyển sang khu vực khác chứ đừng đứng yên ngay trước mặt kẻ thù. Tương tự, nếu bất kỳ ai nhận thấy sự lúng túng nào từ đồng đội bên cạnh họ thì tốt nhất nên cố gắng hỗ trợ thay vì bỏ lại phía sau. Bằng việc liên lạc với nhau thông qua tin tức, chẳng hạn như hàng rào dự báo bóng hay dấu chân trái đạo quân đỏ, nó rất hữu ích để tránh mắc lỗi cơ bản do thiếu chú ý hoặc bối cảnh chung. Cố gắng giao tiếp r

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 23.3 MB/s eta 0:00:00
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2025.4.7
    Uninstalling unsloth-2025.4.7:
      Successfully uninstalled unsloth-2025.4.7
